In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import time
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt
import matplotlib as mp
from matplotlib import patches
%matplotlib inline
from IPython.display import HTML
mp.rcParams['animation.embed_limit'] = 2**128
from matplotlib import animation

import pythreejs as p3js
from copy import deepcopy, copy

from itertools import permutations

This models a n link pendulum, same like the one in JM's lecture, only in 3D. $n$ balls of radius $r$ and mass $m$ are located at the center of each massless rod. An observer of mass $m_o$ may be attached to each ball, its distance rom the center of each ball is $\beta \cdot r$ perpendcular to the massless rod the ball is attached to.\
Collisions between balls are handled with the **Hunt-Crossley** method, details see below.

In order to fix the last point, $P_n$ at $l_n\cdot N.x$ I use the method described below.

*feder method*\
In order to fix $P_n$ at $l_n \cdot N.x$, I introduce a point $Pn$ at the position $P_n$ should be at, and attach spring and, optionally, a dampening device from $P_n$ to $Pn$. The advantage of this approach is that it seems stable over longer times. Of course, it dissipates energy if the dampening is unequal to zero, which the original system does not.


**Symbols used**

- $q_{x1}..q_{zn}:\hspace{78 mm} \text{generalized coordinates}$
- $u_{x1}..u_{zn}:\hspace{78 mm} \text{generalized speeds}$
- $N:\hspace{90 mm} \text{inertial frame}$
- $A_1,..., A_n:\hspace{75 mm} \text{body fixed frames of each each ball}$
- $P_0.....P_n:\hspace{75 mm} \text{points at the end of each rod}$
- $Dmc_i$:\hspace{85 mm} \text{Centers of the balls}
- $Pn:\hspace{89.5 mm} \text{auxiliary point}$
- $l_1, ....., l_n:\hspace{74.5 mm} \text{length of the rods and length from $P_{i-1}$ to $P_i$}$
- $ln:\hspace{90 mm} \text{distance from $P_0$ to $Pn$ along the N.x axis}$
- $m, m_o:\hspace{84 mm} \text{mass of ball, mass of observer}$
- $r, k, d:\hspace{83.5 mm} \text{radius of ball, spring constant, dampening constant}$
- $ k_b: \hspace{90 mm} \text{spring constant of elastic ball}$
- $c_\tau: \hspace{90 mm} \text{experimental constant needed for Hunt-Crossley}$
- $l_0: \hspace{91 mm} \text{natural length of the spring}$
- $aux_{0x}, aux_{0y}, aux_{0z}, aux_{nx}, aux_{ny}, aux_{nz}:\hspace{23 mm} \text{virtual speeds for reaction force}$
- $f_{0x}, f_{0y}, f_{0z}, f_{nx}, f_{ny}, f_{nz}: \hspace{50 mm} \text{reaction forces}$
- $rhodt_{max} \hspace{85 mm} \text{holds the speeds just before impact}$


With *term_info* you can select whether information (dynamic symbols, free symbols, number of operations) of mass matrix, force vector and reaction forces vector (*eingepraegt*) should be printed or not. Not printing speeds up the process of setting up Kane's equations by around 15%...20%\
If *daempfung* is set to TRUE, a speed dependent dampening will be applied in parallel to the spring.\
If *frictionless = True* the balls are ideally slick.


*observation*\
The number of operations increase a lot with increasing number of balls.\
Values below are *without* dampening, dampening increases the size of the force vector by a factor of almost 10, they are also **without friction** during the collisions:
- $n = 2$:
    - mass matrix 4,000 operations
    - force vector 42,000 operations
    - reaction force 25,000 operations
- $n = 3$:
     - mass matrix      70,000 operations
     - force vector    790,000 operations
     - reaction  force 225,000 operation
- $n = 4$:
    - mass matrix 560,000 operations
    - force vector 8,500,000 operations
    - reaction force 1,500,000 operations
    
**With friction** I get these number of operations:
- $n = 3$:
    - mass matrix 70,000
    - force vector 23,000,000
    - reaction force 7,000,000
 
I obviously did not try for larger $n$.
    
For the speed constraints below I need the relationship between $d/dt(q_{ij})$ and $u_{ij}$, with $ i \in \{x, y, z \}$ and $j \in \{1,.. ,n\}$  where $u_{ij}$ are the speeds defined below. It will be stored in *antwort*
For this I solve the linear equation at the very bottom. I use *sm.solve* as the linear system is small, und *sm.solve* returns the solution nicely as a dictionary. For large linear systems, Jason Moore's *LUsolve* method is much faster.\
This is also basically used to get the kinematic equations.

In [ ]:
start = time.time()
start5 = None # just to avoid an error in the pythreejs animation, if the matplotlib animation has not run.

# here are n balls, and (n+1) frames, including the inertial frame N
#===================
n = 3
#===================
daempfung    = False
#===================
term_info    = True
#===================
frictionless = True
#===================

if n < 2 or isinstance(n, int) == False:
    raise Exception("n must be an integer >= 2")


aux0x, aux0y, aux0z, auxnx, auxny, auxnz = me.dynamicsymbols('aux0x, aux0y, aux0z, auxnx, auxny, auxnz')
f0x, f0y, f0z, fnx, fny, fnz = me.dynamicsymbols('f0x, f0y, f0z, fnx, fny, fnz')
aux = [aux0x, aux0y, aux0z, auxnx, auxny, auxnz]
F = [f0x, f0y, f0z, fnx, fny, fnz]
rhs = [sm.symbols('rhs' + str(i)) for i in range(3*n, 6*n)]


N = me.ReferenceFrame('N')
P0 = me.Point('P0')
P0.set_vel(N, aux0x*N.x + aux0y*N.y + aux0z*N.z)

A = [N]
Dmc = []
Po = []
P = [P0]
q1 = []
u1 = []
l = []

for i in range(1, n+1):
    P.append(me.Point('P' + str(i)))
    Dmc.append(me.Point('Dmc' + str(i)))
    Po.append(me.Point('Po' + str(i)))
    A.append(me.ReferenceFrame('A' + str(i)))

    q1.append([me.dynamicsymbols('q' + k + str(i)) for k in ('x', 'y', 'z')])
    u1.append([me.dynamicsymbols('u' + k + str(i)) for k in ('x', 'y', 'z')])
    
    l.append(sm.symbols('l' + str(i)))
    
    
m, mo, g, r, k, kb, d, l0, ln, beta, ctau, mu = sm.symbols('m, mo, g, r, k, kb, d, l0, ln, beta, ctau, mu')
iXX, iYY, iZZ, t = sm.symbols('iXX, iYY, iZZ, t')
rhodtmax = [sm.symbols('rhodt_' + str(i) + str(j)) for i, j in permutations(range(n), r=2)]

rot  = []    # used for the kinematic equations below.
rot1 = []   # dto.

for i in range(1, n+1):
    A[i].orient_body_fixed(A[i-1], (q1[i-1][1], q1[i-1][0], q1[i-1][2]), '213')
    rot.append(A[i].ang_vel_in(A[0]))
    A[i].set_ang_vel(A[i-1], u1[i-1][0]*A[i].x + u1[i-1][1]*A[i].y + u1[i-1][2]*A[i].z)
    rot1.append(A[i].ang_vel_in(A[0]))
    
    P[i].set_pos(P[i-1], l[i-1]*A[i].y)
    P[i].v2pt_theory(P[i-1], A[0], A[i])
    
    Dmc[i-1].set_pos(P[i-1], l[i-1]/2.*A[i].y)
    Dmc[i-1].v2pt_theory(P[i-1], A[0], A[i])
    
    Po[i-1].set_pos(Dmc[i-1], beta*r*A[i].x)
    Po[i-1].v2pt_theory(Dmc[i-1], A[0], A[i])
    
Pn = me.Point('Pn')
Pn.set_pos(P[0], ln * A[0].x)
Pn.set_vel(A[0], auxnx*A[0].x + auxny*A[0].y + auxnz*A[0].z)


gleichung = []
for i in range(n):
    for uv in A[i+1]:
        gleichung.append(me.dot(rot[i]-rot1[i], uv))
variablen = [sm.Derivative(j, t) for i in range(n) for j in q1[i]]

antwort = sm.solve(gleichung, variablen)
zaehler = 0.
for zahl in variablen:
    zaehler += antwort[zahl].count_ops(visual=False)
print(f'antwort contains {zaehler:.0f} operations')

This is needed to get initial generalized coordinates and speeds compatible with the configuration constraint and the resulting speed constraints.\
Here the dictionary *antwort* is needed.\
lnitial generalized coordinates and generalized speeds, which are compatible with the configuration constraint and the resulting speed constraints are found numerically before the integration starts.

In [ ]:
distanz = [P[i+1].pos_from(P[i]) for i in range(n)]
hol  = sum(distanz) - ln*A[0].x
holm = me.dot(hol, hol)

holx = me.dot(hol, A[0].x)
holy = me.dot(hol, A[0].y)
holz = me.dot(hol, A[0].z)

hol_mat = sm.Matrix([holx, holy, holz])
        
holxdt = holx.diff(t).subs(antwort)
holydt = holy.diff(t).subs(antwort)
holzdt = holz.diff(t).subs(antwort)

speed_mat = sm.Matrix([holxdt, holydt, holzdt])

matrix_A = speed_mat.jacobian((u1[-2][2], u1[-1][0], u1[-1][2]))
vector_b = speed_mat.subs({u1[-2][2]: 0., u1[-1][0]: 0., u1[-1][2]: 0.})
loesung  = matrix_A.LUsolve(-vector_b)

if term_info == True:
    print('loesung DS', set().union(*[me.find_dynamicsymbols(loesung[i]) for i in range(3)]))
    print('loesung has {} operations'.format(sum([loesung[i].count_ops(visual=False) for i in range(3)])))

This function calculates the **forces** and the **torques** when two balls collide.

1.\
The impact force is on the line $\overline{P_1 P_2}$. I use Hunt Crossley's method to calculate it\
It's direction is $P_1 \to P_2$.\
2.\
The friction force acting on the contact point $CP_2$ is proportional to the component of relative speeds of the contact points lying in the plane perpendicular to $\overline{P_1 P_2}$, the impact force and a friction factor $m_{u}$.\
It is equivalent to a force through $P_2$ and to a torque acting on $A_2$, the ball's fixed frame.


**Note about the force during the collisions**

 **Hunt Crossley's method**
 
My reference is this article, given to me by JM\
https://www.sciencedirect.com/science/article/pii/S0094114X23000782 \

 
This is with dissipation during the collision, the general force is given in (63) as\
$f_n = k_0 \cdot \rho + \chi \cdot \dot \rho$, with $k_0$ as above, $\rho$ the penetration, and $\dot\rho$ the speed of the penetration.\
In the article it is stated, that $n = \frac{3}{2}$ is a good choice, it is derived in Hertz' approach. Of course, $\rho, \dot\rho$ must be the signed magnitudes of the respective vectors.

A more realistic force is given in (64) as:\
$f_n = k_0 \cdot \rho^n + \chi \cdot \rho^n\cdot \dot \rho$, as this avoids discontinuity at the moment of impact.

**Hunt and Crossley** give this value for $\chi$, see table 1:

$\chi = \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{k_0}{\dot \rho^{(-)}}$, 
where $c_\tau = \dfrac{v_1^{(+)} - v_2^{(+)}}{v_1^{(-)} - v_2^{(-)}}$, where $v_i^{(-)}, v_i^{(+)}$ are the speeds of $body_i$, before and after the collosion, see (45), $\dot\rho^{(-)}$ is the speed right at the time the impact starts. $c_\tau$ is an experimental factor, apparently around 0.8 for steel.

Using (64), this results in their expression for the force:

$f_n = k_0 \cdot \rho^n \left[1 + \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{\dot\rho}{\dot\rho^{(-)}}\right]$

with $k_0 = \frac{4}{3\cdot(\sigma_1 + \sigma_2)} \cdot \sqrt{\frac{R_1 \cdot R_2}{R_1 + R_2}}$, where $\sigma_i = \frac{1 - \nu_i^2}{E_i}$, with $\nu_i$ = Poisson's ratio, $E_i$ = Young"s modulus, $R_1, R_2$ the radii of the colliding bodies, $\rho$ the penetration depth. All is near equations (54) and (61) of this article.

As per the article, $n = \frac{3}{2}$ is always to be used.

*spring energy* =   $ k_0 \cdot \int_{0}^{\rho} k^{3/2}\,dk$ = $k_0 \cdot\frac{2}{5} \cdot \rho^{5/2}$\
I assume, the dissipated energy cannot be given in closed form, at least the article does not give one.

*Note*  
$c_\tau = 1.$ gives **Hertz's** solution to the impact problem, also described in the article.


**Friction when a ball hits a ball**

This website:
https://math.stackexchange.com/questions/2195047/solve-the-vector-cross-product-equation

gives: $b = a \times x \rightarrow x = \dfrac{b \times a}{|a|^2}$

This way, I can easily get the force acting on $P_2$ without any further geometric considerations. Of course the friction force has opposite direction to the relative speeds\
The friction force on $CP_2$  is equivalent to a force on $P_2$ (center of the ball) and a torque on $A_2$ (ball fixed frame)


In [ ]:
def HC_disc(N, A1, A2, P1, P2, r, ctau, rhodtmax, k0):
    '''
This function returns the forces, torques on P2, when colliding with P1.
Very similar to HC_wall above
    '''    
    vektor = P2.pos_from(P1)
    richtung = vektor.normalize()
    abstand  = vektor.magnitude()
    rho = 2. * r - abstand       # positive in the ball has penetrated the wall
    CP01 = me.Point('CP01')
    CP01.set_pos(P1, 0.5 * abstand * richtung)
    vCP01 = CP01.v2pt_theory(P1, N, A1)
    rhodt = me.dot(vCP01, richtung)             #me.dot(P1.pos_from(P0).diff(t, N), richtung)
    rho = sm.Max(rho, sm.S(0))
    
    forcec = k0 * rho**(3/2) * ( 1. + 3./2. * (1 - ctau) * rhodt / rhodtmax) * (richtung *
            sm.Heaviside(2. * r - abstand, 0.))
    
    CP02 = me.Point('CP02')
    CP02.set_pos(P2, -0.5 * abstand * richtung)
    vCP02 = CP02.v2pt_theory(P2, N, A2)
    
    
    friction_force = forcec.magnitude() * mu * -(CP02.vel(N) - CP01.vel(N))        
    hilfs = CP02.pos_from(P2)
    torque = hilfs.cross(friction_force) * sm.Heaviside(2. * r - abstand, 0.)
    forcef = 1./me.dot(hilfs, hilfs) * torque.cross(hilfs) * sm.Heaviside(2. * r - abstand, 0.)
        
    return [forcec, forcef, torque]

**Body list and list of forces for Kane's equations**

If I assume friction between the balls, that is *frictionless = False*, the force vector created by Kane's equations below becomes unmanagably large. My iPad ran out of memory when trying to count the operations in it.\
The dampening (of the attachment of $P_n$ to $Pn$) is proportional to the speed of $P_n$ relative to $Pn$, which is fixed at $l_n \cdot N$.x. However, only the component of the speed in $P_n Pn$ direction is relevant. I get it using the dot product.\
As it does not seem to help, but drastically blows up the force term of Kane's equations, I do not use it.

As there may be numerical problems if $P_n$ is very close to $Pn$, I use the intermediate variable $test$, instead of using .*magnitude()*

Maybe I do not do this the most efficient way:\
I ignore, that the impact force of $ball_i$ on $ball_j$ is equal in magnitude and opposite in direction of the impact force of $ball_j$ on $ball_i$, and same with friction and with torque. But this way it is easier to keep the indices 'straight'.

In [ ]:
I = [me.inertia(A[i], iXX, iYY, iZZ) for i in range(1, n+1)]

body = [me.RigidBody('body' + str(i+1), Dmc[i], A[i+1], m, (I[i], Dmc[i])) for i in range(0, n)]
obs = [me.Particle('obs' + str(i+1), Po[i], mo) for i in range(0, n)]
BODY = body + obs

# Gravitational forces
punkte = Dmc + Po
massen = [m] * n + [mo] * n
FL1 = [(punkt, -masse*g*A[0].y) for punkt, masse in zip(punkte, massen)] 

# Reaction forces
FL_react = [(P[0], f0x*A[0].x + f0y*A[0].y + f0z*A[0].z), (Pn, fnx*A[0].x + fny*A[0].y + fnz*A[0].z)]

# forces due to the balls hitting each other. My assumption is that never will 3 balls hit each other
# at the same time.
FB = []
distanz_Dmc = [Dmc[i].pos_from(Dmc[j]).magnitude() for i, j in permutations(range(n), r=2)]
rhodtmax1 = [me.dot(Dmc[i].pos_from(Dmc[j]).diff(t, N).subs(antwort), Dmc[i].pos_from(Dmc[j]).normalize()) 
            for i, j in permutations(range(n), r=2)]
print('rhodtmax1 DS', set.union(*[me.find_dynamicsymbols(rhodtmax1[j]) 
                    for j in range(len(list(permutations(range(n), r=2))))]))
zaehler = -1

for i, j in permutations(range(n), r=2):
    zaehler += 1
    FB.append((Dmc[i], HC_disc(N, A[j+1], A[i+1], Dmc[j], Dmc[i], r, ctau, rhodtmax[zaehler], kb)[0]))
    if frictionless == False:
        FB.append((Dmc[i], HC_disc(N, A[j+1], A[i+1], Dmc[j], Dmc[i], r, ctau, rhodtmax[zaehler], kb)[1]))
        FB.append((A[i+1], HC_disc(N, A[j+1], A[i+1], Dmc[j], Dmc[i], r, ctau, rhodtmax[zaehler], kb)[2]))

# Forces due to the spring from P_n to Pn
PPF  = Pn.pos_from(P[n])
PPFx = sm.Abs(me.dot(PPF, A[0].x))
PPFy = sm.Abs(me.dot(PPF, A[0].y))
PPFz = sm.Abs(me.dot(PPF, A[0].z))
PPF_len = sm.Abs(sm.sqrt(PPFx**2 + PPFy**2 + PPFz**2))

test = 1. / sm.Max(PPF_len, 1.e-20)   # to avoid divisions by zero
FL2 = [(P[n], -k * (l0 - PPF_len) * PPF * test), (Pn, k * (l0 - PPF_len) * PPF * test)]

#friction forces
# Adding this force term increased the number of operations in force by a factor of around 10.
if daempfung == True:
    PPFdt = PPF.diff(t, N)
    PPFdt_wirk = me.dot(PPFdt, PPF * test ) # only the speed in PPF direction is relevant
    FL3 = [(P[n], d * PPFdt_wirk * PPF * test), (Pn, -d * PPFdt_wirk * PPF * test)]
else:
    FL3 = []
FL = FL1 + FL2 + FL3 + FL_react + FB

**Kane's equations**

Note, that I do **not** use velocity constraints, but use the spring to enforce the configuration constraint.\
I tried velocity constraints before I came up with the spring, but the numerical integration would only work over short times. I do not know, why this is so.

Note the frames used for setting up the kinematic equations. Using other frames may drastically increase the number of operations in the *force vektor*.

In [ ]:
# q1, u1 need to be 'flattened' for use in Kane's set up.
q1h = [q1[i][j] for i in range(0, len(q1)) for j in range(3)]
u1h = [u1[i][j] for i in range(0, len(u1)) for j in range(3)]

q_ind = q1h
u_ind = u1h
kd = [me.dot(rot[i] - rot1[i], uv) for i in range(n) for uv in A[i+1]]
#raise Exception()
KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd, u_auxiliary=aux)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
if term_info == True:
    print('MM DS', me.find_dynamicsymbols(MM))
    print('MM free symbols', MM.free_symbols)
    print('MM contains {} operations'.format(sum([MM[i, j].count_ops(visual=False) for i in range(MM.shape[0]) for j in range(MM.shape[1])])), '\n')

force = KM.forcing_full.subs({i: sm.S(0.) for i in aux})
if term_info == True:
    print('force DS', me.find_dynamicsymbols(force))
    print('force free symbols', force.free_symbols)
    print('force contains {} operations'.format(sum([force[i].count_ops(visual=False) for i in range(force.shape[0])])), '\n')

print('it took {:.3f} sec to get Kanes equations'.format(time.time() - start))

Calculate the **reaction forces**\
For some reason the virtual speeds appear in *eingepraegt*. As per Jason Moore's advise, I set them to zero.

In [ ]:
start1 = time.time()
eingepraegt_dict = {sm.Derivative(i, t): j for i, j in zip(u1h, rhs)}
eingepraegt = (KM.auxiliary_eqs.subs(eingepraegt_dict)).subs({i: sm.S(0) for i in aux})
if term_info == True:
    print('shape of eingepraegt', eingepraegt.shape)
    print('eingepraegt DS', me.find_dynamicsymbols(eingepraegt))
    print('eingepraegt free symbols', eingepraegt.free_symbols)
    print('eingepraegt contains {} operations'.format(sum([eingepraegt[i].count_ops(visual=False) for i in range(eingepraegt.shape[0])])), '\n')

print('it took {:.3f} sec to get the reaction forces'.format(time.time() - start1))

Always good to look at the energies of a system. Absent any friction, the total energy must be constant (a necessary, but of course not a sufficient condition). This has helped me often to correct the set up of Kane's equations.\
The virtual speeds in aux must be set to zero to for the kinetic energy.

- *spring_energie:*$\hspace{50 mm} \text{energy stored in the spring connecting $P_n$ to $Pn$}$
- *spring2_energie:*$\hspace{48 mm} \text{energy stored in the elastic balls during a collission}$

P4pos used to check the accuracy of the location of $P_n$

In [ ]:
kin_dict = {i: 0. for i in aux}
kin_energie = sum([koerper.kinetic_energy(N).subs(kin_dict) for koerper in BODY])
pot_energie = sum([masse*g*me.dot(teil.pos_from(P[0]), N.y) for teil, masse in zip(punkte, massen)])
spring_energie = 0.5 * k * (l0 -  PPF_len)**2

spring2_energie = 0.
for i, j in permutations(range(n), r=2):
    abstand = Dmc[i].pos_from(Dmc[j]).magnitude()
    rho = sm.Max(2.*r - abstand, sm.S(0))
    spring2_energie += 2./5. * kb * rho**(5/2) * sm.Heaviside(rho, sm.S(0)) 
spring2_energie = spring2_energie / 2.
              
P4pos = [me.dot(Pn.pos_from(P[n]), uv) for uv in N]

Convert the sympy functions to numpy functions to do numeric calculations. *cse=True* speeds up the numerical integration a lot (in this case, without *cse=True* it would not be practical on my machine), it may slow down the lambdification.

In [ ]:
start1 = time.time()
qL  = q1h + u1h
pL  = [m, mo, g, r, k, kb, d, l0, ln, beta, mu, ctau] + l + [iXX, iYY, iZZ] + rhodtmax
pL1 = [m, mo, g, r, k, kb, d, l0, ln, beta, mu, ctau] + l + [iXX, iYY, iZZ]

# q_ind1, q_dep1, u_ind1, up_dep1 are needed further down in the lambdification 
u_ind1 = deepcopy(u1h) # deepcopy needed to avoid 'destroying' u1h
hilfs1 = u_ind1.pop(-1)
hilfs2 = u_ind1.pop(-2)
hilfs3 = u_ind1.pop(-2) 
u_dep1 = [hilfs3, hilfs2, hilfs1]

q_ind1 = deepcopy(q1h)
hilfs1 = q_ind1.pop(-1)
hilfs2 = q_ind1.pop(-2)
hilfs3 = q_ind1.pop(-2)
q_dep1 = [hilfs3, hilfs2, hilfs1]

MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

print(' it took {:.3f} sec for the lambdification of force and of MM'.format(time.time() - start1))

In [ ]:
eingepraegt_lam = sm.lambdify(F + qL + pL + rhs, eingepraegt, cse=True)

# find initial values of the generalized coordinates which satisfy the configuration constraint
holm_lam = sm.lambdify(q1h + pL, holm, cse=True)

# to get the initial values of the dependent speeds
speed_mat1_lam = sm.lambdify(u_dep1 + q1h + u_ind1 + pL, speed_mat, cse=True)
speed_mat11_lam = sm.lambdify(qL + pL, speed_mat, cse=True)

loesung_lam = sm.lambdify(q1h + u_ind1 + pL, loesung, cse=True)

kin_lam = sm.lambdify(qL + pL, kin_energie, cse=True)
pot_lam = sm.lambdify(qL + pL, pot_energie, cse=True)
spring_lam = sm.lambdify(qL + pL, spring_energie, cse=True)
spring2_lam = sm.lambdify(qL + pL, spring2_energie, cse=True)

loc_lam = sm.lambdify(q_dep1 + q_ind1 + pL, hol_mat, cse=True)

crash_lam = sm.lambdify(q1h + pL, distanz_Dmc, cse=True)

matrix_A_lam = sm.lambdify(q1h + u_ind1 + pL, matrix_A, cse=True)
P4pos_lam = sm.lambdify(qL + pL, P4pos, cse=True)

rhodtmax1_lam   = sm.lambdify(qL + pL1, rhodtmax1, cse=True)
distanz_Dmc_lam = sm.lambdify(qL + pL1, distanz_Dmc, cse=True)

**Numerical Integration**

1.\
In the case of a 3D link, it is not so trivial to guess initial conditions which meet the configuration constraint. I try to find good approximations by minimizing the length of the configuration constraint.
Then this initial result is improved by numerically by solving the configuration constraint for three generalized coordinates. (This n link pendulum has $ 3 \cdot n -3$ independent generalized coordinates and 3 dependent coordinates)\
There are "many" feasible initial conditions to meet the configuration constraint. Different initial guesses generally lead to dfferent solutions minimizing the length of the configuration constraint. Not all of them lead to 'reasonable' results. I assume these are numerical problems with such large equations - but of course I do not know. 

2.\
The initial generalized coordinates must be such that the balls are separated.\
if they are not, an *Exception* will be raised.

3.\
I use the speed constraints to get initial generalized speeds which are consistent.


4.\
The speed constraints are fulfilled better if $n = 3$ or $n = 4$ than with $n = 2$.

5.\
Initial speeds much different from zero may cause problems.

All else is standard.

In [ ]:
#============================
# Input variables
# the names are the same as in setting up Kane's equations, except that a "1" is appended.
#============================
m1  = 1.e0
mo1 = 1.e-1
g1  = 9.8
r1  = 1.
k1  = 1.e7
kb1 = 1.e4    
d1  = 1.e3
l01 = 1.e-7

mu1  = 0.
ctau1 = 0.8


l1 = [4. + 1. * (i+1) for i in range(len(l))]
ln1 = np.sum(l1) / 2.5
print("length of the rods:", l1)
print("distance from P0 to Pn, that is ln:", ln1, "\n")
beta1 = .95

u11 = [(-1.)**i * 0.  for i in range(len(u1h))]
rhodtmax1 = [1. for i, j in permutations(range(n), r=2)]
intervall = 25.
#=====================================================================================
schritte = int(intervall * 3000)
times = np.linspace(0., intervall, schritte)

iXX1 = 2./5. * m1 * r1**2    #from the internet.
iYY1, iZZ1 = iXX1, iXX1

pL_vals  = [m1, mo1, g1, r1, k1, kb1, d1, l01, ln1, beta1, mu1, ctau1
    ] + l1 +  [iXX1, iYY1, iZZ1] + rhodtmax1
pL1_vals = [m1, mo1, g1, r1, k1, kb1, d1, l01, ln1, beta1, mu1, ctau1] + l1 +  [iXX1, iYY1, iZZ1]

def func(x0, args):
    return holm_lam(*x0, *args)

# find good consistent initial generalized coordinates
x0 = [1.] * len(q1h) # initial guess. Different guesses may lead to different solutions.
for i in range(10):
    anfangs_q = minimize(func, x0, pL_vals)
    anfangs_q = list(anfangs_q.x)
    x0 = anfangs_q
print(f"config constraint violated by initial guess: {f'{holm_lam(*anfangs_q, *pL_vals):0.3e}':>20}")

# improve the initial generalized coordi8nates.
def func2(x0, args):
    return loc_lam(*x0, *args).reshape(3)

anfangs_q1 = deepcopy(anfangs_q) # anfangs_q1 may be manipulated without affecting anfangs_q
hilfs1 = anfangs_q1.pop(-1)
hilfs2 = anfangs_q1.pop(-2)
hilfs3 = anfangs_q1.pop(-2)

args = anfangs_q1 + pL_vals
x0 = [hilfs3, hilfs2, hilfs1]
AAA = fsolve(func2, x0, args)
anfangs_q[-4] = AAA[0]
anfangs_q[-3] = AAA[1]
anfangs_q[-1] = AAA[2]
print(f"after improvement, config constraint violated by: {f'{holm_lam(*anfangs_q, *pL_vals):0.3e}':>15}", '\n')

# Setting the natural length of the spring a bit smaller that the initial error of the location of
# P_n from Pn may help.
pL_vals[7] = holm_lam(*anfangs_q, *pL_vals) / 2.

# avoid initial conditions, where the balls are not separated from eacb other
zaehler = -1
for i, j in permutations(range(n), r=2):
    if i < j:
        zaehler += 1
        print(f"initial distance Dmc_{i} from Dmc_{j} is {f'{crash_lam(*anfangs_q, *pL_vals)[zaehler]:0.3f}':>5}")
if np.min(crash_lam(*anfangs_q, *pL_vals)) <= 2. * r1:
    raise Exception('Some balls are not separated')

def func1(x0, args):
    return speed_mat1_lam(*x0, *args).reshape(3)

anfangs_u_ind = deepcopy(u11)
anfangs_u_ind.pop(-1)
anfangs_u_ind.pop(-2)
anfangs_u_ind.pop(-2)

hilfs1, hilfs2, hilfs3 = loesung_lam(*anfangs_q, *anfangs_u_ind, *pL_vals)
hilfs1 = hilfs1[0]
hilfs2 = hilfs2[0]
hilfs3 = hilfs3[0]
anfangs_u_dep = [hilfs1, hilfs2, hilfs3]

args1 = anfangs_q + anfangs_u_ind + pL_vals
print('\n', 'Violation of speed constraints:', [f'{speed_mat1_lam(*anfangs_u_dep, *args1)[i][0]:0.3e}'
                    for i in range(3)], "\n")

matrix_A1_det = np.linalg.det(matrix_A_lam(*anfangs_q, *anfangs_u_ind, *pL_vals))
print(f'Determinante of matrix_A, used to determine u_dep = {matrix_A1_det:.3f} \n')

u11[-4] = hilfs1
u11[-3] = hilfs2
u11[-1] = hilfs3
y0 = anfangs_q + u11 
print('initial conditions are:', '\n', 'y0 =', [f'{y0[i]:.3f}' for i in range(len(y0))], '\n')

The actual **numerical integration** starts here.

In [ ]:
start2 = time.time()
t_span = (0., intervall)
def gradient(t, y, args):

    '''
Here I try to find the speed just before a collission takes place.
    '''
    zaehler  = -1
    laenge = -len(list(permutations(range(n), r=2)))
    for i, j in permutations(range(n), r=2):
        zaehler += 1 
        if 0. < 2.*r1 - distanz_Dmc_lam(*y, *pL1_vals)[zaehler] < 0.001:
            args[laenge] = rhodtmax1_lam(*y, *pL1_vals)[zaehler]
        laenge += 1

    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]
        
resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), method='Radau')
#   , max_step=max_step)        
#    ,atol = 1.e-5, rtol = 1.e-5)

resultat = resultat1.y.T
print('resultat shape', resultat.shape, '\n')
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ', message is:', resultat1.message, '\n')

print("To numerically integrate an intervall of {} sec the routine cycled {} times and it took {:.3f} sec "
      .format(intervall, resultat1.nfev, time.time() - start2), '\n')

Plot whichever generalized coordinates you want to see.

In [ ]:
schritte1 = resultat.shape[0]
times1 = times[0: schritte1]
bezeichnung = [str(i) for i in qL]

fig, ax = plt.subplots(figsize=(10,5))
for i in (10, 13, 16):
    ax.plot(times1, [resultat[l, i] for l in range(resultat.shape[0])], label=bezeichnung[i])
ax.set_xlabel('time (sec)')
if daempfung == True:
    ax.set_title('Selected coordinates, with friction = {}'.format(d1))
else:
    ax.set_title('Selected coordinates')
ax.legend();   

Plot the errors in the location of $P_n$ from the desired location, that is from $l_n \cdot N$.x

In [ ]:
schritte1 = resultat.shape[0]
times1 = times[0: schritte1]

P4x_np = np.empty(schritte1)
P4y_np = np.empty(schritte1)
P4z_np = np.empty(schritte1)

for i in range(schritte1):
    P4x_np[i] = P4pos_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[0]
    P4y_np[i] = P4pos_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[1]
    P4z_np[i] = P4pos_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[2]
    
    
fig, ax = plt.subplots(figsize=(10,5))
for i, j in zip((P4x_np, P4y_np, P4z_np), ('P_n X', 'P_n Y', 'P_n Z')):
    ax.plot(times1, i, label=j)
ax.set_xlabel('time (sec)')
if daempfung == True:
    print('Error in P[n] coordinates, friction = {}'.format(d1))
else:
    ax.set_title('Error in P[n] coordinates')
ax.legend();
mean_average = np.sqrt(np.mean(P4x_np)**2 + np.mean(P4y_np)**2 + np.mean(P4z_np)**2)
print('average distance of Pn from required position:', mean_average)
print('average standard deviation                    ', (np.std(P4x_np) + np.std(P4y_np)
                    + np.std(P4z_np)) / 3.)

Plot the **energies** of the system. Absent any friction, that is $c_{tau} = 1$ and $m_u = 0.$ it should be zero.\
Of course, the system will continue to dissipate energy if dampening is unequal to zero, but as this blows up the equations dramatically, I do not use it.\

In [ ]:
schritte1 = resultat.shape[0]
times1 = times[0: schritte1]

kin_np = np.empty(schritte1)
pot_np = np.empty(schritte1)
spring_np = np.empty(schritte1)
spring2_np = np.empty(schritte1)
total_np = np.empty(schritte1)

for i in range(schritte1):
    kin_np[i]     = kin_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    pot_np[i]     = pot_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    spring_np[i]  = spring_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    spring2_np[i] = spring2_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    total_np[i]   = kin_np[i] + pot_np[i] + spring_np[i] + spring2_np[i]
    
fig, ax = plt.subplots(figsize=(10,5))
for i, j in zip((kin_np, pot_np, spring_np, spring2_np, total_np), 
    ('kin. energy', 'pot. energy', 'spring energy', 'spring2 energy', 'total energy')):
    ax.plot(times1, i, label=j)
ax.set_xlabel('time (sec)')
if daempfung == True:
    ax.set_title('Energy of system, spring constant = {}, dampening constant = {}'.format(k1, d1))
else:
    ax.set_title('Energy of the system')
ax.legend();
    
if d1 == 0.:
    maxtotal = np.max(total_np)
    mintotal = np.min(total_np)
    print('max deviation of total energy from being constant is {:.3f} % of max. total energy'.
              format(np.abs((maxt - mint)/maxt) * 100))
else:
    maxtotal = np.max(total_np)
    mintotal = np.min(total_np)
    print('Total energy dissipated is {:.3f} Nm'.format(maxtotal - mintotal))   
            

# find when the balls collided, which balls collided, and whatthe the energy of the collission 
zusammenstoss = np.empty((schritte1, 3))
zusammenstoss1 = []
for i, zeit in enumerate(times1):
    boxen = crash_lam(*[resultat[i, k] for k in range(int(resultat.shape[1]/2.))], *pL_vals)
    zusammenstoss[i] = (0., 0., 0.)
    for k_1 in range(int(len(list(permutations(range(n), r=2))))):
        if boxen[k_1] <= 2. * r1:
            staerke = spring2_lam(*[resultat[i, l] for l in range(resultat.shape[1])], *pL_vals)
            zusammenstoss[i] = (k_1, zeit, staerke)

for i in range(schritte1 - 1):
    if zusammenstoss[i, 1] == 0. and zusammenstoss[i+1, 1] != 0.:
        zusammenstoss1.append(zusammenstoss[i+1])
        staerke_max = -1.
        k_1 = i + 1
        while k_1 < schritte1 and zusammenstoss[k_1, 1] != 0.:
            staerke_max = max(staerke_max, zusammenstoss[k_1, 2])
            k_1 += 1
        zusammenstoss1[-1][2] = staerke_max

print('\n')
print('When did which balls collide, and what was the energy of the collision?', '\n')
bezeichnung = ["ball" + str(j) + " collided with ball" + str(i) for i, j in permutations(range(n), r=2)]
for i in range(len(zusammenstoss1)):
    k_1, t11, energie = zusammenstoss1[i][0], zusammenstoss1[i][1], zusammenstoss1[i][2]
    k_1 = int(k_1)
    print('at {:.2f} sec {} with energy around {:.0f} Nm'.format(t11, bezeichnung[k_1], energie))
print('in total {} collisions took place'.format(len(zusammenstoss1)))   

fig, ax = plt.subplots(figsize=(10,5))
ax.plot(times1, spring_np)
ax.set_xlabel('time (sec)')
ax.set_title('Energy in the spring used to pull P[n] towards Pn')
print('\n')
print(f'Average energy stored in the spring used to pull P[n] towards Pn is {np.mean(spring_np):.5f} Nm')

Plot the **hysteresis curves** of $ball_i$ colliding with $ball_j$\
The red numbers on the curves indicate the time of the impact.

In [ ]:
zaehler1 = -1
for i1, j1 in permutations(range(n), r=2):
    HC_kraft = []
    HC_displ = []
    HC_times = []
    zaehler  = 0
    i0 = 0
    zaehler1 += 1
    for i in range(resultat.shape[0]):
        abstand = 2.*r1  - distanz_Dmc_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
            *pL1_vals)[zaehler1]
        if abstand < 0.:
            i0 = i+1

        if abstand >= 0. and i0 == i:
            walldt = rhodtmax1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
                                        *pL1_vals)[zaehler1]
        if abstand >= 0.:
            rhodt = rhodtmax1_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
                                      *pL1_vals)[zaehler1]
            kraft0 = kb1 * abstand**(3/2) * (1. + 3./2. * (1 - ctau1) * rhodt/walldt)
            HC_displ.append(abstand)
            HC_kraft.append(kraft0)
            HC_times.append((zaehler, times[i]))
            zaehler +=1

    HC_displ = np.array(HC_displ)
    HC_kraft = np.array(HC_kraft)
# print only, if there were collisions.    
    if len(HC_displ) != 0 and i1 < j1:
        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(HC_displ, HC_kraft, color='green')
        ax.set_xlabel('penetration depth (m)')
        ax.set_ylabel('contact force (N)')
        ax.set_title(f'hysteresis curves of successive impacts of ball_{i1} with ball_{j1}, ctau = {ctau1}, mu = {mu1}')
    
        zeitpunkte = 10
        reduction = max(1, int(len(HC_times)/zeitpunkte))
        for k in range(len(HC_times)):
            if k % reduction == 0:
                coord  = HC_times[k][0]
                ax.text(HC_displ[coord], HC_kraft[coord], f'{HC_times[k][1]:.2f}', color="red")


*Calculate the reaction forces on $P_0$ and $Pn$.*

- numerically solve $rhs = MM^{-1} \cdot force$
- numerically solve $eingepraegt = 0.$ for the reaction forces in $F$. I iterate here 3 times, sometimes this improves the accuracy of the solution.
- plot the reaction forces

I need many time points to get 'nice' hysteresis curves. This makes this part very slow. I limit the points considered to around *zeitpunkte*. 

In [ ]:
times1 = []
resultat1 = []

#=======================
zeitpunkte = 25000
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times1.append(times[i])
        resultat1.append(resultat[i])

schritte1 = len(times1)
resultat1 = np.array(resultat1)
times1 = np.array(times1)
print('number of points considered:',len(times1))

# RHS calculated numerically, too large to do it symbolically. Needed for reaction forces.
RHS1 = np.zeros((schritte1, resultat.shape[1]))
for i in range(schritte1):
    RHS1[i, :] = np.linalg.solve(MM_lam(*[resultat[i, j]for j in range(resultat.shape[1])], *pL_vals), 
        force_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
        *pL_vals)).reshape(resultat.shape[1])

#calculate implied forces numerically
def func (x, *args):
# just serves to 'modify' the arguments for fsolve.
    return eingepraegt_lam(*x, *args).reshape(6)

kraft1x = np.empty(schritte1)
kraft1y = np.empty(schritte1)
kraft1z = np.empty(schritte1)
kraft5x = np.empty(schritte1)
kraft5y = np.empty(schritte1)
kraft5z = np.empty(schritte1)

x0 = tuple((1., 1., 1., 1., 1., 1.))   # initial guess

for i in range(schritte1):
    for _ in range(3):
        y0 = [resultat[i, j] for j in range(resultat.shape[1])]
        rhs1 = [RHS1[i, j] for j in range(3*n, 6*n)]
        args = tuple(y0 + pL_vals + rhs1)
        AF = fsolve(func, x0, args=args).reshape(6)
        x0 = tuple(AF)      # improved guess. Should speed up convergence of fsolve
    kraft1x[i] = AF[0]
    kraft1y[i] = AF[1]
    kraft1z[i] = AF[2]
    kraft5x[i] = AF[3]
    kraft5y[i] = AF[4]
    kraft5z[i] = AF[5]
    
fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(times1, kraft1x, label='force in X direction')
plt.plot(times1, kraft1y, label='force in Y direction')
plt.plot(times1, kraft1z, label='force in Z direction')
ax.set_xlabel('time (sec)')
ax.set_ylabel('force (N)')
ax.set_title('Reaction Forces on P0')
plt.legend();

fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(times1, kraft5x, label='force in X direction')
plt.plot(times1, kraft5y, label='force in Y direction')
plt.plot(times1, kraft5z, label='force in Z direction')
ax.set_xlabel('time (sec)')
ax.set_ylabel('force (N)')
ax.set_title('Reaction Forces on Pn')
plt.legend();

Plot the motion of the system as a projection onto the X / Z plane. The large circles are the balls, the smaller circles are the points $P_1...P_n$. Their color indicates their height above the lowest point.\
The balls are shown 'correctly' in that the 'darker' ball is shown to be 'under' the lighter ball. I did not do this with the points $P_1....P_n$, just to many cases to distinguish.

I need many time points to get nice hysteresis curves. This makes the animation very slow. I limit the time points considered to around *zeitpunkte*.

In [ ]:
start4 = time.time()

Dmc_loc = [[me.dot(punkt.pos_from(P[0]), uv) for uv in N] for punkt in Dmc]
Po_loc  = [[me.dot(punkt.pos_from(P[0]), uv) for uv in N] for punkt in Po]
Pdc = copy(P)
Pdc.pop(0)
P_loc   = [[me.dot(punkt.pos_from(P[0]), uv) for uv in N] for punkt in Pdc]

Dmc_lam = sm.lambdify(qL + pL, Dmc_loc, cse = True)
Po_lam = sm.lambdify(qL + pL, Po_loc, cse = True)
P_lam = sm.lambdify(qL + pL, P_loc, cse = True)

times1 = []
resultat1 = []

#=======================
zeitpunkte = 500
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times1.append(times[i])
        resultat1.append(resultat[i])

schritte1 = len(times1)
resultat1 = np.array(resultat1)
times1 = np.array(times1)
print('number of points considered:',len(times1))


Dmc_np = np.empty((schritte1, n, 3))
Po_np  = np.empty((schritte1, n, 3))
P_np   = np.empty((schritte1, n, 3))

# generate the data for the animation
for i in range(schritte1):
    for j in range(n):
        for k_1 in range(3):
            Dmc_np[i, j, k_1] = Dmc_lam(*[resultat1[i, l] for l in range(resultat1.shape[1])], *pL_vals)[j][k_1]
            Po_np[i, j, k_1]  = Po_lam(*[resultat1[i, l] for l in range(resultat1.shape[1])], *pL_vals)[j][k_1]
            P_np[i, j, k_1]   = P_lam(*[resultat1[i, l] for l in range(resultat1.shape[1])], *pL_vals)[j][k_1]

# determine the size of the animation
minx = min(np.min(Dmc_np[:, :, 0].flatten()), np.min(P_np[:, :, 0].flatten()))
minz = min(np.min(Dmc_np[:, :, 2].flatten()), np.min(P_np[:, :, 2].flatten()))
maxx = max(np.max(Dmc_np[:, :, 0].flatten()), np.max(P_np[:, :, 0].flatten()))
maxz = max(np.max(Dmc_np[:, :, 2].flatten()), np.max(P_np[:, :, 2].flatten()))
skala = max(np.abs(minx), np.abs(maxx), np.abs(minz), np.abs(maxz))

# get the color scale. For some reason, integers seem to be needed. In order to have a finer scale,
# I multiply the heights of the points / balls by 100. before converting them to integers
CDmc = np.empty((schritte1, n), int)
CP   = np.empty((schritte, n), int)
for i in range(schritte1):
    for k_1 in range(n):
        CDmc[i, k_1] = int(Dmc_np[i, k_1, 1] * 100.)
        CP[i, k_1]   = int(P_np[i, k_1, 1] * 100.)
        
ymin = min(np.min(CDmc.flatten()), np.min(CP.flatten()))
ymax = max(np.max(CDmc.flatten()), np.max(CP.flatten()))

# This is to asign colors of 'plasma' to the points.
Test = mp.colors.Normalize(ymin, ymax)
Farbe = mp.cm.ScalarMappable(Test, cmap='plasma')

farbeDmc = [Farbe.to_rgba(CDmc[0, k]) for k in range(n)] # colprs of starting positions
farbeP = [Farbe.to_rgba(CP[0, k]) for k in range(n)]


def animate_pendulum(times, Dmc_np, Po_np, P_np):
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'aspect': 'equal'})
    fig.colorbar(Farbe, label='Height of the bodies/pointd above lowest level, factor = 100 \n The actual heights are the numbers divided by factor', shrink=0.9, ax=ax)

    ax.axis('on')
    ax.set_xlim(-skala - 2., skala + 2.)
    ax.set_ylim(-skala - 2., skala + 2.)
    ax.set_xlabel('X axis', fontsize=13)
    ax.set_ylabel('Z axis', fontsize=13)
    
    LINE1 = [] # connection of P_i to P_i+1
    for i in range(n):  
        LINE1.append(ax.plot([], [], 'bo', linestyle = '-', linewidth=0.5, markersize=1)) 
    
    LINE2 = [] # observer on ball_i
    for i in range(n):
        LINE2.append(ax.plot([], [], color='red', marker='o', markersize=5)) 
    
    ax.plot([0., 0.], [0., 0.], color='green', marker='o', markersize=6) # P0, fixed point
    
    kreisDL = []       # the balls   
    kreisPL = []       # the points
    for i in range(n):
        kreisD = patches.Circle((Dmc_np[0, i, 0], Dmc_np[0, i, 2]), radius = r1, fill=True, 
                                color=farbeDmc[i], ec='black')  # balls
        ax.add_patch(kreisD)
    
        kreisP = patches.Circle((P_np[0, i, 0], P_np[0, i, 2]), radius = r1/4., fill=True, 
                                color=farbeP[i], ec='black')  # observers
        ax.add_patch(kreisP)
        
        kreisDL.append(kreisD)
        kreisPL.append(kreisP)

    def animate(i):        
        ax.set_title('Projection of the motion onto the X / Z plane. \n Red dots are the particles. \n The green dot is P0 \n Running time {:.2f} sec'.format(times[i]), fontsize=13)
        
# this is to ensure that the lower balls are shown to be below the higher balls.
        ordnung  = [Dmc_np[i, k] for k in range(n)]
        ordnung1 = sorted(ordnung, key = lambda test: test[1])
        farbe_ordnung = [int(ordnung1[k][1] * 100) for k in range(n)]
        farbeD = [Farbe.to_rgba(farbe_ordnung[k]) for k in range(n)]
        
        for j, kreis in enumerate(kreisDL):
            kreis.set_center((ordnung1[j][0], ordnung1[j][2]))
            kreis.set_color(farbeD[j])
            kreis.set_ec("black")
            
        for j, kreis in enumerate(kreisPL):
            kreis.set_center((P_np[i, j, 0], P_np[i, j, 2]))
            kreis.set_color(Farbe.to_rgba(CP[i, j]))
            kreis.set_ec("black")
            
# for whatever reason, the ax.plot(....) ist stored in LINE1 / LINE2 as a one-element-list, no directly/
# hence linie[0] is needed. Same applies for the return.
        for j, linie in enumerate(LINE1):
            if j == 0:
                x_values = [0., P_np[i, 0, 0]]
                z_values = [0., P_np[i, 0, 2]]
            else:
                x_values = [P_np[i, j-1, 0], P_np[i, j, 0]]
                z_values = [P_np[i, j-1, 2], P_np[i, j, 2]]             
            linie[0].set_data(x_values, z_values)                  
        
        for j, linie in enumerate(LINE2):
            linie[0].set_data(Po_np[i, j, 0], Po_np[i, j, 2])
                                
        return *LINE1[0], *LINE2[0]

    anim = animation.FuncAnimation(fig, animate, frames=schritte1,
                                   interval=1000*max(times1) / schritte1,
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times1, Dmc_np, Po_np, P_np)
start5 = time.time()
print('it took {:.3f} sec to run the animation, before HTML(..)'.format(start5 - start4))
HTML(anim.to_jshtml())    # needed, when run on an iPad, I know no other way to do it. It is SLOW!

Animation using pythreejs.
This is basically copied from a program by Jason Moore, just adapted to my needs here.

NOTE: the 'reference frame' for pythreejs seems to be:
- X - axis downwards, color red
- Y - axis to the right, color green (hence:)
- Z - axis pointing to the observer, color blue

Rotation is used to transform my coordinate system used above to set up the equations of motion 
to the one prescribed by pythreejs.

If you know from the beginning, that you want to use pythreejs it is probably better to use its
orientation of coordinates, when setting up Kane's equations. Saves the trouble of guessing, which
rotation is correct. I am not sure my rotation is fully correct, just played around until it 'looked' reasonable.

If you want to draw a line between two point, only way I know is CylinderGeometry. In this case, the two points must be on the Y axis of the coordinate system, because the length of the CylinderGeometry extends in this direction. (there may be other ways to do it, I do not know them.)

The points in time are large to get nice hysteresis curves. This will cause the animation to take a long time. I reduce the number of points in time to around *zeitpunkte*.

In [ ]:
start6 = time.time()

times1 = []
resultat1 = []

#=======================
zeitpunkte = 500
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times1.append(times[i])
        resultat1.append(resultat[i])

schritte1 = len(times1)
resultat1 = np.array(resultat1)
times1 = np.array(times1)
print('number of points considered:',len(times1))

Dmc_loc = [[me.dot(punkt.pos_from(P[0]), uv) for uv in N] for punkt in Dmc]
punkt_loc = [[me.dot(punkt.pos_from(P[0]), uv) for uv in N] for punkt in Po]
Dmc_loc_lam = sm.lambdify(qL + pL, Dmc_loc, cse=True)
punkt_loc_lam = sm.lambdify(qL + pL, punkt_loc, cse=True)


winkel = sm.symbols('winkel')
Rotation1 = sm.Matrix([[sm.cos(winkel), -sm.sin(winkel), 0], [sm.sin(winkel), sm.cos(winkel), 0], [0., 0., 1]])
Rot_lam = sm.lambdify(winkel, Rotation1.T)
Rotation = Rot_lam(np.pi/2.)

TC_store = []
TR_store = []
TP_store = []
body_mesh_store = []
track_store = []
farben = ['orange', 'blue', 'green', 'yellow', 'red']

laengen = l1

# the list of frames A contains the inertial frame A[0] = N. This must not be here
AB = copy(A) # for whatever reason, deepcopy does not work here
AB.pop(0)

for i in range(n):
#for its mass center
    TC = sm.eye(4)
    TC[:3, :3] = (AB[i].dcm(N)) * Rotation
    TC = TC.reshape(16, 1)
    TC_lam = sm.lambdify(qL + pL, TC)   

    TR = sm.eye(4)
    TR[:3, :3] = (AB[i].dcm(N)) * Rotation
    TR = TR.reshape(16, 1)
    TR_lam = sm.lambdify(qL + pL, TR) 
    
    TP = sm.eye(4)
    TP[:3, :3] = (AB[i].dcm(N)) * Rotation
    TP = TP.reshape(16, 1)
    TP_lam = sm.lambdify(qL + pL, TP) 


# store the information about the body, expressed in TAc for every time step.
    TCs = []   # for the ball
    TRs = []   # for the rod
    TPs = []   # for the red dot
    
# Create the TAs, containing 'one TA' for each time step
# resultat contains the results of the numeric integration. 
# where the numeric integration was evaluated
    for k_1 in range(resultat1.shape[0]):
        zeit = times[i]
        TCi = TC_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)  # the balls
        TRi = TR_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)  # the rod
        TPi = TP_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)  # the dot

# TAi[12], TAi[13], TAi[14] hold the location of A[i] w.r.t. N.
# As the axis chosen for solving the equations of motion, and the axis given by pythreejs do not 
# coincide, the values for TAi[..] must be permutated accordingly.
# of course here different locations for center of ball and center of mass.
        TRi[12] = -Dmc_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][1]
        TRi[13] =  Dmc_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][0]
        TRi[14] =  Dmc_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][2]
    
        TCi[12] = -Dmc_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][1]
        TCi[13] =  Dmc_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][0]
        TCi[14] =  Dmc_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][2] 

        TPi[12] = -punkt_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][1]
        TPi[13] =  punkt_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][0]
        TPi[14] =  punkt_loc_lam(*[resultat1[k_1, l] for l in range(resultat1.shape[1])], *pL_vals)[i][2]

        TRs.append(TRi.squeeze().tolist())
        TCs.append(TCi.squeeze().tolist())
        TPs.append(TPi.squeeze().tolist())
       
    TC_store.append(TCs)
    TR_store.append(TRs)
    TP_store.append(TPs)
    
# Create the objects, which will move
# 1. The ball
    body_geom_C = p3js.SphereGeometry(r1, 12, 12)
    body_material_C = p3js.MeshStandardMaterial(color=farben[i], wireframe=False)
    body_mesh_C = p3js.Mesh(geometry=body_geom_C, material=body_material_C, name='ball_' + str(i))

# 2. Rod
    body_geom_R = p3js.CylinderGeometry(radiusTop=0.05, radiusBottom=0.05, height=laengen[i], 
                    radialSegments=6, heightSegments=10, openEnded=False)
    body_material_R = p3js.MeshStandardMaterial(color='black', wireframe=False)
    body_mesh_R = p3js.Mesh(geometry=body_geom_R, material=body_material_R, name='rod_' + str(i))
    
# 3. the dot    
    body_geom_P = p3js.SphereGeometry(0.25, 12, 12)
    body_material_P = p3js.MeshStandardMaterial(color='red', wireframe=False)
    body_mesh_P = p3js.Mesh(geometry=body_geom_P, material=body_material_P, name='punkt_' + str(i))

# locate the body in 3D space and add the coordinate system of the body
    body_mesh_R.matrixAutoUpdate = False
    body_mesh_R.add(p3js.AxesHelper(0.1))  # length of the axis of the ball system A2
    body_mesh_R.matrix = TR_store[i][0]             # starting point of the animation

    body_mesh_C.matrixAutoUpdate = False
    body_mesh_C.add(p3js.AxesHelper(0.01))    # length of the axis of the center of mass system A2
    body_mesh_C.matrix = TC_store[i][0]          # starting point of the animation
    
    body_mesh_P.matrixAutoUpdate = False
    body_mesh_P.add(p3js.AxesHelper(0.01))    # length of the axis of the center of mass system A2
    body_mesh_P.matrix = TP_store[i][0]          # starting point of the animation
    
    
    body_mesh_store.append(body_mesh_C)
    body_mesh_store.append(body_mesh_R)
    body_mesh_store.append(body_mesh_P)
    

# Create the 'picture'.
# all the 'paramters' are taken by trial and error.
view_width = 1200
view_height = 400

# Values just found by trial an error.
p1, p2 = 5, 5
if n == 2:
    p3 = 18
elif n == 3:
    p3 = 22
else:
    p3 = 30

camera = p3js.PerspectiveCamera(position=[p1, p2, p3],
                                up=[-1.0, 0.0, 0.0],
                                aspect=view_width/view_height)

key_light = p3js.DirectionalLight(position=[0, 0, 10])
ambient_light = p3js.AmbientLight()

axes = p3js.AxesHelper(20)

children = []
for i in range(3*n):
    children = children + [body_mesh_store[i], axes, camera, key_light, ambient_light]

scene = p3js.Scene(children=children)
controller = p3js.OrbitControls(controlling=camera)
renderer = p3js.Renderer(camera=camera, scene=scene, controls=[controller],
                         width=view_width, height=view_height)

# Create the action, simply copied from JM's lecture.

for i in range(n):
    eigenname = 'ball_'+str(i)
    track_C = p3js.VectorKeyframeTrack(
        name="scene/" + eigenname + ".matrix",
        times=times1,
        values=TC_store[i])
    
    eigenname = 'rod_' + str(i)
    track_R = p3js.VectorKeyframeTrack(
        name="scene/" + eigenname + ".matrix",
        times=times1,
        values=TR_store[i])
    
    eigenname = 'punkt_' + str(i)
    track_P = p3js.VectorKeyframeTrack(
        name="scene/" + eigenname + ".matrix",
        times=times1,
        values=TP_store[i])
    
    track_store += [track_C] + [track_R] + [track_P]

duration = times1[-1] - times1[0]
clip = p3js.AnimationClip(tracks=track_store, duration=duration)
action = p3js.AnimationAction(p3js.AnimationMixer(scene), clip, scene)

# avoid error if the animation has not run before this runs.
if start5 != None:
    print(' it tooks {:.3f} sec to run the complete program'.format(time.time() - start6 + start5 - start ))

renderer

This will start the animation.\
As I know nothing about pythreejs, I cannot explain how it works.\
For some reason, with python 3.11 some error is raised, which did not happen with python 3.9. This error does not seem to affect the animation.

In [ ]:
action